In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
pd.set_option('display.max_columns', 500)
plotly.offline.init_notebook_mode(connected=True)

In [2]:
df_confirmed=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
df_deaths=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
df_newest=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-19-2020.csv')

In [3]:
#convert table
def converttable(df):
    cols=df.columns.tolist()
    pd_list=[]
    for i in range(5,df.shape[1]):
        temp_cols=cols[:4]
        temp_cols.append(cols[i])
    # print(temp_cols)
        temp_pd=df[temp_cols].copy()
        temp_pd['dt']=cols[i]
        temp_pd.rename(columns={cols[i]:'value'},inplace=True)
        pd_list.append(temp_pd)
    df_new=pd.concat(pd_list,axis=0,ignore_index=True)
    return df_new

In [4]:
df_confirmed_new = converttable(df_confirmed)
df_deaths_new = converttable(df_deaths)

In [5]:
#dt format
df_confirmed_new['dt']=pd.to_datetime(df_confirmed_new['dt'])
df_deaths_new['dt']=pd.to_datetime(df_deaths_new['dt'])

In [6]:
# Rename cols
df_confirmed_new.rename(columns = {'value': 'confirmed'}, inplace = True)
df_deaths_new.rename(columns = {'value': 'deaths'}, inplace = True)

In [7]:
# Merge two tables
df_merged = df_confirmed_new.merge(df_deaths_new[['Province/State', 'Country/Region', 'dt', 'deaths']], 
                                   on = ['Province/State', 'Country/Region', 'dt'], 
                                   how = 'inner')
df_merged = df_merged[['Province/State', 'Country/Region', 'Lat', 'Long', 'dt', 'confirmed', 'deaths']]
#df_merged.head()

In [8]:
# Extract US data (includes cruise ship)
df_us = df_merged[df_merged['Country/Region'] == 'US'].copy()
df_merged.drop(index = df_us.index, inplace = True)

# Aggregate
us_agg = df_us.groupby(by = 'dt').sum().copy()
us_agg.reset_index(inplace = True)
us_agg['Lat'] = 37.0902
us_agg['Long'] = -95.7129
us_agg['Province/State'] = np.nan
us_agg['Country/Region'] = 'US'

# Add row to general table with US data
df_merged = pd.concat([df_merged, us_agg], sort = False, ignore_index = True)

In [9]:
# Extract US data
#df_us = df_merged[df_merged['Country/Region'] == 'US'].copy()
# Drop US data from main table
#df_merged.drop(index = df_us.index, inplace = True)

# Extract counties
#df_us_county = df_us[df_us['Province/State'].str.find('County')>0].copy()
# Drop counties from US data
#df_us.drop(index = df_us_county.index, inplace = True)

# Extract cities
#df_us_city = df_us[df_us['Province/State'].str.find(', ')>0].copy()
# Drop cities from US data
#df_us.drop(index = df_us_city.index, inplace = True)

# Check what is left
#df_us['Province/State'].unique()

# Drop others
#df_us.drop(index = df_us[df_us['Province/State'] == 'Diamond Princess'].index, inplace = True)
#df_us.drop(index = df_us[df_us['Province/State'] == 'Grand Princess'].index, inplace = True)

#df_us['Province/State'].unique().shape[0]

In [10]:
# Aggregate data for all US states 
#us_agg = df_us.groupby(by = 'dt').sum().copy()
#us_agg.reset_index(inplace = True)
#us_agg['Lat'] = 37.0902
#us_agg['Long'] = -95.7129
#us_agg['Province/State'] = np.nan
#us_agg['Country/Region'] = 'US'

# Add row to general table with US data
#df_merged = pd.concat([df_merged, us_agg], sort = False, ignore_index = True)

In [11]:
# Extract and aggregate data for other countries

# China
df_china = df_merged[df_merged['Country/Region'] == 'China'].copy()
df_merged.drop(index = df_china.index, inplace = True)
china_agg = df_china.groupby(by = 'dt').sum()
china_agg.reset_index(inplace = True)
china_agg['Lat'] = 35.8617
china_agg['Long'] = 104.1954
china_agg['Province/State'] = np.nan
china_agg['Country/Region'] = 'China'
df_merged = pd.concat([df_merged, china_agg], sort = False, ignore_index = True)

# Canada
df_canada = df_merged[df_merged['Country/Region'] == 'Canada'].copy()
df_merged.drop(index = df_canada.index, inplace = True)
canada_agg = df_canada.groupby(by = 'dt').sum()
canada_agg.reset_index(inplace = True)
canada_agg['Lat'] = 56.1304
canada_agg['Long'] = -106.3468
canada_agg['Province/State'] = np.nan
canada_agg['Country/Region'] = 'Canada'
df_merged = pd.concat([df_merged, canada_agg], sort = False, ignore_index = True)

# Australia
df_aus = df_merged[df_merged['Country/Region'] == 'Australia'].copy()
df_merged.drop(index = df_aus.index, inplace = True)
aus_agg = df_aus.groupby(by = 'dt').sum()
aus_agg.reset_index(inplace = True)
aus_agg['Lat'] = -25.2744
aus_agg['Long'] = 133.7751
aus_agg['Province/State'] = np.nan
aus_agg['Country/Region'] = 'Australia'
df_merged = pd.concat([df_merged, aus_agg], sort = False, ignore_index = True)

# France
df_fra = df_merged[df_merged['Country/Region'] == 'France'].copy()
df_merged.drop(index = df_fra.index, inplace = True)
fra_agg = df_fra.groupby(by = 'dt').sum()
fra_agg.reset_index(inplace = True)
fra_agg['Lat'] = 46.2276
fra_agg['Long'] = 2.2137
fra_agg['Province/State'] = np.nan
fra_agg['Country/Region'] = 'France'
df_merged = pd.concat([df_merged, fra_agg], sort = False, ignore_index = True)

# Denmark
df_den = df_merged[df_merged['Country/Region'] == 'Denmark'].copy()
df_merged.drop(index = df_den.index, inplace = True)
den_agg = df_den.groupby(by = 'dt').sum()
den_agg.reset_index(inplace = True)
den_agg['Lat'] = 56.2639
den_agg['Long'] = 9.5018
den_agg['Province/State'] = np.nan
den_agg['Country/Region'] = 'Denmark'
df_merged = pd.concat([df_merged, den_agg], sort = False, ignore_index = True)

# United Kingdom
df_uk = df_merged[df_merged['Country/Region'] == 'United Kingdom'].copy()
df_merged.drop(index = df_uk.index, inplace = True)
uk_agg = df_uk.groupby(by = 'dt').sum()
uk_agg.reset_index(inplace = True)
uk_agg['Lat'] = 56.2639
uk_agg['Long'] = 9.5018
uk_agg['Province/State'] = np.nan
uk_agg['Country/Region'] = 'United Kingdom'
df_merged = pd.concat([df_merged, uk_agg], sort = False, ignore_index = True)

# Netherlands
df_net = df_merged[df_merged['Country/Region'] == 'Netherlands'].copy()
df_merged.drop(index = df_net.index, inplace = True)
net_agg = df_net.groupby(by = 'dt').sum()
net_agg.reset_index(inplace = True)
net_agg['Lat'] = 52.1326
net_agg['Long'] = 5.2913
net_agg['Province/State'] = np.nan
net_agg['Country/Region'] = 'Netherlands'
df_merged = pd.concat([df_merged, net_agg], sort = False, ignore_index = True)

In [12]:
# Check if duplications country/date
df_merged[df_merged.duplicated( subset = ['dt', 'Country/Region'], keep = False) == True]

,Province/State,Country/Region,Lat,Long,dt,confirmed,deaths


In [13]:
# Drop column province
df_merged.drop(columns = ['Province/State'], inplace = True)

In [14]:
# Active cases
#df_merged['Active'] = df_merged['confirmed'] - df_merged['deaths'] - df_merged['recovered']

# Mortality rate
df_merged['MortalityRate'] = df_merged['deaths']/df_merged['confirmed']

In [15]:
# Days since

# Sort dataset by date ascending
df_merged.sort_values(by = 'dt', ascending = True, inplace = True)

# List of countries
country_list = df_merged['Country/Region'].unique().tolist()

# Locate day first case for each country
first_confirmed = []
first_50confirmed = []
first_10deaths = []
    
for c in country_list:
    tmp = df_merged[df_merged['Country/Region'] == c].copy()
    first_confirmed.append(tmp.loc[tmp.confirmed.ne(0).idxmax(), 'dt'])
    first_50confirmed.append(tmp.loc[(tmp.confirmed >= 50).idxmax(), 'dt'])
    first_10deaths.append(tmp.loc[(tmp.deaths >= 10).idxmax(), 'dt'])

df_first = pd.DataFrame(data = {'Country/Region': country_list, 
                                'first_confirmed': first_confirmed,
                                'first_50confirmed': first_50confirmed,
                                'first_10deaths': first_10deaths,
                               } 
                       )

# Convert to datetime
df_first['first_confirmed'] = pd.to_datetime(df_first['first_confirmed'])
df_first['first_50confirmed'] = pd.to_datetime(df_first['first_50confirmed'])
df_first['first_10deaths'] = pd.to_datetime(df_first['first_10deaths'])

#df_first.head()

In [16]:
# Merge firsts with main table
df_merged = df_merged.merge(df_first, on = 'Country/Region', how = 'left')

# Convert data type
df_merged['dt'] = pd.to_datetime(df_merged['dt'])

# Calculate days since
df_merged['days_since_1st_conf'] = df_merged['dt'] - df_merged['first_confirmed']
df_merged['days_since_50th_conf'] = df_merged['dt'] - df_merged['first_50confirmed']
df_merged['days_since_10th_deaths'] = df_merged['dt'] - df_merged['first_10deaths']

# Transform date units
df_merged['days_since_1st_conf'] = df_merged['days_since_1st_conf'].dt.days
df_merged['days_since_50th_conf'] = df_merged['days_since_50th_conf'].dt.days
df_merged['days_since_10th_deaths'] = df_merged['days_since_10th_deaths'].dt.days

# Drop intermediate columns
df_merged.drop(columns = ['first_confirmed', 'first_50confirmed', 'first_10deaths'], inplace = True)

#df_merged[df_merged['Country/Region'] == 'Germany'][['dt', 'confirmed', 'deaths', 'days_since_1st_conf',
#                                                     'days_since_50th_conf', 'days_since_10th_deaths']]

In [17]:
# New cases

# Sort values
df_merged.sort_values(by = ['Country/Region', 'dt'], ascending = True, inplace = True)

# Differences
df_merged['confirmed_newcases'] = df_merged.groupby(by = ['Country/Region']).confirmed.diff()
df_merged['deaths_newcases'] = df_merged.groupby(by = ['Country/Region']).deaths.diff()

#df_merged[df_merged['Country/Region'] == 'Germany'][['dt', 'confirmed', 'confirmed_diff']]

In [29]:
# New cases moving average
df_merged['confirmed_newcases_movavg'] = np.nan
df_merged['deaths_newcases_movavg'] = np.nan

for c in country_list:
    
    tmp = df_merged[df_merged['Country/Region'] == c].copy()
    tmp = tmp.sort_values(by = 'dt').reset_index()
    
    for i in range(1, days_num-1):
        
        df_merged.loc[tmp.loc[i,'index'], 'confirmed_newcases_movavg'] = ((
            tmp.loc[i-1, 'confirmed_newcases'] +
            tmp.loc[i, 'confirmed_newcases'] +
            tmp.loc[i+1, 'confirmed_newcases'] )/3) 
        
        df_merged.loc[tmp.loc[i,'index'], 'deaths_newcases_movavg'] = ((
            tmp.loc[i-1, 'deaths_newcases'] +
            tmp.loc[i, 'deaths_newcases'] +
            tmp.loc[i+1, 'deaths_newcases'] )/3)    
        

# Check
#df_merged[df_merged['Country/Region'] == 'Germany'][['dt', 'confirmed', 'confirmed_newcases', 
#                                                     'confirmed_newcases_growth', 
#                                                     'confirmed_newcases_growth_movavg']]

In [18]:
# Open population table
df_pop = pd.read_csv('world_pop_by_country.csv')
df_pop.rename(columns = {'Country Name': 'Country/Region', '2018': 'population_2018'}, inplace = True)

# Merge population table
df_merged = df_merged.merge(df_pop, on = 'Country/Region', how = 'left')

df_merged.sort_values('dt').head()

,Country/Region,Lat,Long,dt,confirmed,deaths,MortalityRate,days_since_1st_conf,days_since_50th_conf,days_since_10th_deaths,confirmed_newcases,deaths_newcases,Country Code,population_2018
0,Afghanistan,33.0000,65.0000,2020-01-23,0.0,0.0,NaN,-32,0,0,NaN,NaN,AFG,37172386.0
854,Barbados,13.1939,-59.5432,2020-01-23,0.0,0.0,NaN,-54,0,0,NaN,NaN,BRB,286641.0
10309,Trinidad and Tobago,10.6918,-61.2225,2020-01-23,0.0,0.0,NaN,-51,-59,0,NaN,NaN,TTO,1389858.0
1586,Cambodia,11.5500,104.9167,2020-01-23,0.0,0.0,NaN,-4,-57,0,NaN,NaN,KHM,16249798.0
6405,Maldives,3.2028,73.2207,2020-01-23,0.0,0.0,NaN,-45,0,0,NaN,NaN,MDV,515696.0


In [19]:
# Cases by 100.000 hab.
df_merged['confirmed_by100000pop'] = df_merged['confirmed']*100000/df_merged['population_2018']
df_merged['deaths_by100000pop'] = df_merged['deaths']*100000/df_merged['population_2018']

# Drop intermediate columns
df_merged.drop(columns = ['Country Code', 'population_2018'], inplace = True)

In [20]:
# Growth rates

df_merged.head()

,Country/Region,Lat,Long,dt,confirmed,deaths,MortalityRate,days_since_1st_conf,days_since_50th_conf,days_since_10th_deaths,confirmed_newcases,deaths_newcases,confirmed_by100000pop,deaths_by100000pop
0,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,-32,0,0,NaN,NaN,0.0,0.0
1,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,-31,1,1,0.0,0.0,0.0,0.0
2,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,-30,2,2,0.0,0.0,0.0,0.0
3,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,-29,3,3,0.0,0.0,0.0,0.0
4,Afghanistan,33.0,65.0,2020-01-27,0.0,0.0,NaN,-28,4,4,0.0,0.0,0.0,0.0


In [21]:
# New cases growth (new cases today divided by new cases yesterday)

df_merged['confirmed_newcases_growth'] = np.nan
df_merged['deaths_newcases_growth'] = np.nan

days_num = df_merged['dt'].unique().shape[0]

for c in country_list:
    
    tmp = df_merged[df_merged['Country/Region'] == c]
    tmp = tmp.sort_values(by = 'dt').reset_index()
    
    for i in range(1, days_num):
        if (tmp.loc[i-1, 'confirmed_newcases'] != 0):
            df_merged.loc[tmp.loc[i,'index'] , 'confirmed_newcases_growth'] = (
                tmp.loc[i, 'confirmed_newcases']/tmp.loc[i-1, 'confirmed_newcases'])
        else:
            df_merged.loc[tmp.loc[i,'index'] , 'confirmed_newcases_growth'] = np.nan
        if (tmp.loc[i-1, 'deaths_newcases'] != 0):
            df_merged.loc[tmp.loc[i,'index'] , 'deaths_newcases_growth'] = (
                tmp.loc[i, 'deaths_newcases']/tmp.loc[i-1, 'deaths_newcases'])
        else:
            df_merged.loc[tmp.loc[i,'index'] , 'deaths_newcases_growth'] = np.nan
        
#df_merged[df_merged['Country/Region'] == 'Germany'][['dt', 'confirmed', 'confirmed_newcases', 
#                                                     'confirmed_newcases_growth']]

In [22]:
# New cases growth moving average

df_merged['confirmed_newcases_growth_movavg'] = np.nan
df_merged['deaths_newcases_growth_movavg'] = np.nan

for c in country_list:
    
    tmp = df_merged[df_merged['Country/Region'] == c].copy()
    tmp = tmp.sort_values(by = 'dt').reset_index()
    
    for i in range(1, days_num-1):
        
        df_merged.loc[tmp.loc[i,'index'], 'confirmed_newcases_growth_movavg'] = ((
            tmp.loc[i-1, 'confirmed_newcases_growth'] +
            tmp.loc[i, 'confirmed_newcases_growth'] +
            tmp.loc[i+1, 'confirmed_newcases_growth'] )/3) 
        
        df_merged.loc[tmp.loc[i,'index'], 'deaths_newcases_growth_movavg'] = ((
            tmp.loc[i-1, 'deaths_newcases_growth'] +
            tmp.loc[i, 'deaths_newcases_growth'] +
            tmp.loc[i+1, 'deaths_newcases_growth'] )/3)    
        

# Check
#df_merged[df_merged['Country/Region'] == 'Germany'][['dt', 'confirmed', 'confirmed_newcases', 
#                                                     'confirmed_newcases_growth', 
#                                                     'confirmed_newcases_growth_movavg']]

In [24]:
# Select data for plotting

# Data for the top 10 countries in number of confirmed cases
last_date = df_merged['dt'].max()
top10 = df_merged[df_merged['dt'] == last_date].sort_values(by = 'confirmed', ascending = False).iloc[0:10, :] 
top10_country = top10['Country/Region'].values

In [25]:
# Plotting colors
top10_col = ['blue', 'orange', 'green', 'crimson', 'grey', 'turquoise', 'yellowgreen', 'steelblue', 'purple',
             'pink'
            ]
title_col = 'navy'
subtitle_col = 'grey'
label_col = 'dimgray'
#bg_col =
#grid_col =

# Plottoing fonts & sizes
title_font = 'PT Sans Narrow'
subtitle_font = 'PT Sans Narrow'
label_font = 'PT Sans'
title_size = 24
subtitle_size = 18
label_size = 14

In [23]:
kkkkk

NameError: name 'kkkkk' is not defined

In [ ]:
# Plot timeline confirmed top ten countries

# Plot

data = []
for i, c in enumerate(top10_country):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].dt,
                           y = df_merged[df_merged['Country/Region'] == c].confirmed,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           hoverlabel = dict(bordercolor = top10_col[i], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10_col[i])),
                          ))
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].dt,
                           y = df_merged[df_merged['Country/Region'] == c].deaths,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           hoverlabel = dict(bordercolor = top10_col[i], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10_col[i])),
                           showlegend = False,
                           visible = False
                          ))

conf_visible = [True, False, True, False, True, False, True, False, True, False, 
                True, False, True, False, True, False, True, False, True, False] 
death_visible = [False, True, False, True, False, True, False, True, False, True, 
                 False, True, False, True, False, True, False, True, False, True]

lay = go.Layout(width = 800, 
                height = 500, 
                #bargap = 0.2,
                xaxis = dict(title='Date',
                             nticks = 10,
                             rangemode = 'nonnegative',
                             zeroline = False,
                            ),
                yaxis = dict(title='Number of Cases',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.7),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Cases for the current 10 most affected countries',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'Subtitle here',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                            ],
                updatemenus=[dict(buttons=list([dict(args = ["yaxis.type", 'linear'],
                                                     label = 'Linear',
                                                     method = 'relayout'
                                                    ),
                                                dict(
                                                    args = ["yaxis.type", 'log'],
                                                    label = 'Logarithmic',
                                                    method = 'relayout'
                                                )
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 1.00,
                                  yanchor = 'top'
                                 ),
                             dict(buttons=list([dict(args = [{'visible': conf_visible, 'showlegend' : True }],
                                                     label = 'Confirmed   ',
                                                     method = 'restyle'
                                                    ),
                                                dict(args = [{'visible': death_visible, 'showlegend' : True }],
                                                     label = 'Deaths',
                                                     method = 'restyle'
                                                    ),
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 0.88,
                                  yanchor = 'top'
                                 ),
                            ]
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)

In [ ]:
# Plot timeline confirmed top ten countries

# Plot

data = []
for i, c in enumerate(top10_country):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].days_since_1st_conf,
                           y = df_merged[df_merged['Country/Region'] == c].confirmed,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           hoverlabel = dict(bordercolor = top10_col[i], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10_col[i])),
                          ))
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].days_since_1st_conf,
                           y = df_merged[df_merged['Country/Region'] == c].deaths,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           hoverlabel = dict(bordercolor = top10_col[i], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10_col[i])),
                           showlegend = False,
                           visible = False
                          ))

conf_visible = [True, False, True, False, True, False, True, False, True, False, 
                True, False, True, False, True, False, True, False, True, False] 
death_visible = [False, True, False, True, False, True, False, True, False, True, 
                 False, True, False, True, False, True, False, True, False, True]

lay = go.Layout(width = 800, 
                height = 500, 
                #bargap = 0.2,
                xaxis = dict(title='Days since 1st case confirmed',
                             nticks = 10,
                             rangemode = 'nonnegative',
                             zeroline = False,
                            ),
                yaxis = dict(title='Number of Cases',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.7),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Cases for the current 10 most affected countries',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'Subtitle here',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                            ],
                updatemenus=[dict(buttons=list([dict(args = ["yaxis.type", 'linear'],
                                                     label = 'Linear',
                                                     method = 'relayout'
                                                    ),
                                                dict(
                                                    args = ["yaxis.type", 'log'],
                                                    label = 'Logarithmic',
                                                    method = 'relayout'
                                                )
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 1.00,
                                  yanchor = 'top'
                                 ),
                             dict(buttons=list([dict(args = [{'visible': conf_visible, 'showlegend' : True }],
                                                     label = 'Confirmed   ',
                                                     method = 'restyle'
                                                    ),
                                                dict(args = [{'visible': death_visible, 'showlegend' : True }],
                                                     label = 'Deaths',
                                                     method = 'restyle'
                                                    ),
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 0.88,
                                  yanchor = 'top'
                                 ),
                            ]
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)

In [ ]:
# Plot mortality rates

# Assign color to top10
top10['color'] = top10_col

# Plot
data = [go.Bar(x = top10.sort_values(by = 'MortalityRate')['MortalityRate'],
               y = top10.sort_values(by = 'MortalityRate')['Country/Region'],
               orientation = 'h',
               name = 'mort_conf',
               marker = dict(color = top10.sort_values(by = 'MortalityRate')['color']),
               hoverinfo = 'text',
               hovertext = ['Confirmed:<br>'+'{:,}'.format(x) for x in top10.sort_values(by = 'MortalityRate')['confirmed']],
               hoverlabel = dict(bordercolor = top10.sort_values(by = 'MortalityRate')['color'], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10.sort_values(by = 'MortalityRate')['color']))
              ),
        go.Bar(x = top10.sort_values(by = 'deaths_by100000pop')['deaths_by100000pop'],
               y = top10.sort_values(by = 'deaths_by100000pop')['Country/Region'],
               orientation = 'h',
               name = 'mort_pop',
               marker = dict(color = top10.sort_values(by = 'deaths_by100000pop')['color']),
               hoverinfo = 'text',
               hovertext = ['Confirmed:<br>'+'{:,}'.format(x) for x in top10.sort_values(by = 'deaths_by100000pop')['confirmed']],
               hoverlabel = dict(bordercolor = top10.sort_values(by = 'deaths_by100000pop')['color'], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10.sort_values(by = 'deaths_by100000pop')['color'])),
               visible = False,
              )
       ]

lay = go.Layout(width = 800, 
                height = 500,
                margin=dict(l=120, r=50, b=100, t=100, pad=4),
                #bargap = 0.2,
                xaxis = dict(title='Mortality Rate',
                             nticks = 10,
                             rangemode = 'nonnegative',
                             zeroline = False,
                             showgrid = True,
                            ),
                yaxis = dict(title='',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.7),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Mortality Rate by Country',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'Subtitle here',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                             dict(x = 1.35,
                                  y = 0.95,
                                  showarrow = False,
                                  text = 'Type of mortality rate:',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = label_font,
                                      size = label_size,
                                      color = label_col,),
                             
                             )
                            ],
                updatemenus=[dict(buttons=list([dict(args = ['visible', [True, False]],
                                                     label = 'Deaths per confirmed case',
                                                     method = 'restyle'
                                                    ),
                                                dict(args = ['visible', [False, True]],
                                                     label = 'Deaths per 100,000 population',
                                                     method = 'restyle'
                                                    ),
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 0.88,
                                  yanchor = 'top'
                                 ),
                            ]
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)

In [27]:
df_merged.head()

,Country/Region,Lat,Long,dt,confirmed,deaths,MortalityRate,days_since_1st_conf,days_since_50th_conf,days_since_10th_deaths,confirmed_newcases,deaths_newcases,confirmed_by100000pop,deaths_by100000pop,confirmed_newcases_growth,deaths_newcases_growth,confirmed_newcases_growth_movavg,deaths_newcases_growth_movavg
0,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,-32,0,0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,-31,1,1,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,-30,2,2,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,-29,3,3,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,33.0,65.0,2020-01-27,0.0,0.0,NaN,-28,4,4,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [30]:
# Plot timeline confirmed top ten countries

# Plot

data = []
for i, c in enumerate(top10_country):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].dt,
                           y = df_merged[df_merged['Country/Region'] == c].confirmed_newcases_movavg,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                          )
               )

lay = go.Layout(width = 800, 
                height = 500, 
                #bargap = 0.2,
                xaxis = dict(title='Days since 1st case confirmed',
                             nticks = 10,
                             rangemode = 'nonnegative',
                             zeroline = False,
                            ),
                yaxis = dict(title='Confirmed cases',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.8),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Confirmed cases for the current 10 most affected countries',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'Subtitle here',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                            ],
                updatemenus=[dict(buttons=list([dict(args = ["yaxis.type", 'linear'],
                                                     label = 'Linear',
                                                     method = 'relayout'
                                                    ),
                                                dict(
                                                    args = ["yaxis.type", 'log'],
                                                    label = 'Logarithmic',
                                                    method = 'relayout'
                                                )
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 1.00,
                                  yanchor = 'top'
                                 ),
                            ]
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)

In [26]:
# Plot growth rates

data = []
for i, c in enumerate(top10_country):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].dt,
                           y = df_merged[df_merged['Country/Region'] == c].deaths_pct_change_movavg,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           
                          )
               )

lay = go.Layout(width = 800, 
                height = 500, 
                #bargap = 0.2,
                xaxis = dict(title='Date',
                             nticks = 10,
                            ),
                yaxis = dict(title='New Cases Confirmed',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.8),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Has the curve flattened?',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'New cases confirmed per day',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                            ],
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)

AttributeError: 'DataFrame' object has no attribute 'deaths_pct_change_movavg'

In [ ]:
# Plot growth rates

data = []
for i, c in enumerate(top10_country):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].dt,
                           y = df_merged[df_merged['Country/Region'] == c].confirmed_diff_movavg,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           hoverlabel = dict(bordercolor = top10_col[i], 
                                 bgcolor = 'white', 
                                 font = dict(color = top10_col[i])),
                          )
               )

lay = go.Layout(width = 800, 
                height = 500, 
                #bargap = 0.2,
                xaxis = dict(title='Date',
                             nticks = 10,
                            ),
                yaxis = dict(title='New Cases Confirmed',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.8),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Has the curve flattened?',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'New cases confirmed each day (3-day average)',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                            ],
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)

In [ ]:
# How is your country doing?
# Plot timeline confirmed top ten countries

# Plot

data = []
for i, c in enumerate(top10_country):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].days_since_1st_conf,
                           y = df_merged[df_merged['Country/Region'] == c].confirmed,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                          ))
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].days_since_1st_conf,
                           y = df_merged[df_merged['Country/Region'] == c].deaths,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           showlegend = False,
                           visible = False
                          ))
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].days_since_1st_conf,
                           y = df_merged[df_merged['Country/Region'] == c].recovered,
                           name = c,
                           marker = dict(color = top10_col[i]),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           showlegend = False,
                           visible = False
                          ))

for i, c in enumerate(df_merged['Country/Region'].unique()):
    data.append(go.Scatter(x = df_merged[df_merged['Country/Region'] == c].days_since_1st_conf,
                           y = df_merged[df_merged['Country/Region'] == c].confirmed,
                           name = c,
                           marker = dict(color = 'black', size = 2.),
                           hoverinfo = 'text+y',
                           hovertext = c,
                           showlegend = False,
                           visible = False
                          ))
    

conf_visible = [True, False, False, True, False, False, True, False, False, True,  False, False, True,  False, False, 
                True, False, False, True, False, False, True, False, False, True, False, False, True, False, False,] 
death_visible = [False, True, False, False, True, False, False, True, False, False, True, False, False, True, False, 
                False, True, False, False, True, False, False, True, False, False, True, False, False, True, False]
recov_visible = [False, False, True, False, False, True, False, False, True, False, False, True, False, False, True, 
                 False, False, True, False, False, True, False, False, True, False, False, True, False, False, True]
    

lay = go.Layout(width = 800, 
                height = 500, 
                #bargap = 0.2,
                xaxis = dict(title='Days since 1st case confirmed',
                             nticks = 10,
                             rangemode = 'nonnegative',
                             zeroline = False,
                            ),
                yaxis = dict(title='Number of Cases',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.7),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'Cases for the current 10 most affected countries',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'Subtitle here',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                            ],
                updatemenus=[dict(buttons=list([dict(args = ["yaxis.type", 'linear'],
                                                     label = 'Linear',
                                                     method = 'relayout'
                                                    ),
                                                dict(
                                                    args = ["yaxis.type", 'log'],
                                                    label = 'Logarithmic',
                                                    method = 'relayout'
                                                )
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 1.00,
                                  yanchor = 'top'
                                 ),
                             dict(buttons=list([dict(args = [{'visible': conf_visible, 'showlegend' : True }],
                                                     label = 'Confirmed   ',
                                                     method = 'restyle'
                                                    ),
                                                dict(args = [{'visible': death_visible, 'showlegend' : True }],
                                                     label = 'Deaths',
                                                     method = 'restyle'
                                                    ),
                                                dict(args = [{'visible': recov_visible, 'showlegend' : True }],
                                                     label = 'Recovered',
                                                     method = 'restyle'
                                                    ),
                                               ]),
                                  direction = "down",
                                  pad = {"r": 10, "t": 10},
                                  showactive = True,
                                  x = 1.025,
                                  xanchor = 'left',
                                  y = 0.88,
                                  yanchor = 'top'
                                 ),
                            ]
               )


fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)


In [ ]:
# The Hubei case

df_hubei = df_china[df_china['Province/State'] == 'Hubei'].copy()
df_hubei['new_cases'] = df_hubei.confirmed.diff()
df_hubei['active'] = df_hubei['confirmed'] - df_hubei['deaths'] - df_hubei['recovered']


data = [go.Scatter(x = df_hubei.dt,
                   y = df_hubei.confirmed,
                   marker = dict(color = 'gray'),
                   name = 'Confirmed',
                   hoverinfo = 'text+y',
                   hovertext = c,
                   hoverlabel = dict(bordercolor = top10_col[i], 
                                     bgcolor = 'white', 
                                     font = dict(color = top10_col[i])),
                  ),
        go.Bar(x = df_hubei.dt,
               y = df_hubei.deaths,
               name = 'Deaths',
               marker = dict(color = 'purple'),
               opacity = 0.7,
               hoverinfo = 'text+y',
                   hovertext = c,
                   hoverlabel = dict(bordercolor = 'orange', 
                                     bgcolor = 'white', 
                                     font = dict(color = 'orange')),
                  ),
        go.Bar(x = df_hubei.dt,
               y = df_hubei.recovered,
               name = 'Recovered',
               marker = dict(color = 'lightseagreen'),
               opacity = 0.7,
               hoverinfo = 'text+y',
                   hovertext = c,
                   hoverlabel = dict(bordercolor = 'orange', 
                                     bgcolor = 'white', 
                                     font = dict(color = 'orange')),
                  ),
        go.Bar(x = df_hubei.dt,
               y = df_hubei.active,
               name = 'Active',
               marker = dict(color = 'orange'),
               opacity = 0.7,
               hoverinfo = 'text+y',
                   hovertext = c,
                   hoverlabel = dict(bordercolor = 'orange', 
                                     bgcolor = 'white', 
                                     font = dict(color = 'orange')),
                  )
        
       ]

lay = go.Layout(width = 1000, 
                height = 500, 
                #bargap = 0.2,
                barmode='stack',
                xaxis = dict(title='Date',
                             nticks = 10,
                             rangemode = 'nonnegative',
                             zeroline = False,
                            ),
                yaxis = dict(title='Number of Cases',
                             type = 'linear',
                            ),
                hovermode = 'closest',
                font = dict(size = label_size,
                            family = label_font,
                            color = label_col,
                           ),
                legend = dict(x = 1.03, 
                              y = 0.7),
                annotations=[dict(x = 0.02,
                                  y = 1.2,
                                  showarrow = False,
                                  text = 'The Hubei Case',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = title_font,
                                      size = title_size,
                                      color = title_col,),
                                 ),
                             dict(x = 0.02,
                                  y = 1.1,
                                  showarrow = False,
                                  text = 'Timeline of events',
                                  xref = 'paper',
                                  yref = 'paper',
                                  font=dict(
                                      family = subtitle_font,
                                      size = subtitle_size,
                                      color = subtitle_col,),
                                 ),
                             dict(x = pd.to_datetime('1/24/20'),
                                  y = 300,
                                  showarrow = True,
                                  text = 'Wuhan lockdown',
                                  xref = 'x',
                                  yref = 'y',
                                  font=dict(
                                      family = label_font,
                                      size = 12,
                                      color = 'navy',),
                                 ),
                             dict(x = pd.to_datetime('2/13/20'),
                                  y = 50000,
                                  showarrow = True,
                                  text = 'Hubei change case definition',
                                  xref = 'x',
                                  yref = 'y',
                                  font=dict(
                                      family = label_font,
                                      size = 12,
                                      color = 'navy',),
                                 ),
                            ],
               )

fig = dict(data=data, layout=lay)
plotly.offline.iplot(fig)


In [ ]:
# Plots to do:

# Growth rates
# Hubei case
# How your country is doing?
# Mortality rate vs. ....

In [ ]:
plot = plotly.offline.plot({'data':data,
                            'layout':lay},
                           include_plotlyjs = False,
                           output_type = 'div',
                           config = dict(showLink = False,
                                         modeBarButtonsToRemove = ['sendDataToCloud'],
                                         displaylogo = False,
                                         responsive = True)
                          )

out_file = open('timeline_confirmed_date.html', 'w')
out_file.write('<!DOCTYPE html><html><head><script src="https://cdn.plot.ly/plotly-latest.min.js"></script></head><body>')
out_file.write(plot)
out_file.write('</body><html>')
out_file.close()

In [ ]:
df_merged.to_csv('df_merged_update.csv')

In [ ]:
#df_confirmed_new.to_csv('confirmed_new.csv')
#df_deaths.to_csv('deaths_new.csv')
#df_recovered.to_csv('recovered.csv')
#df_newest.to_csv('df_newest.csv')

In [ ]:
df_hubei = df_china[df_china['Province/State'] == 'Hubei'].copy()
df_hubei['new_cases'] = df_hubei.confirmed.diff()
df_hubei